In [180]:
import tensorflow as tf
from tensorflow.contrib import learn
import pandas as pd

####
# Helper function:
#  Return the k most frequently appearing keywords in the dataframe
def top_k(data_df, vec, k):
    X = vec.fit_transform(data_df['sms'].values)
    labels = vec.get_feature_names()
    
    return pd.DataFrame(columns = labels, data = X.toarray()).sum().sort_values(ascending = False)[:k]



sms_df = pd.read_csv('spam.csv', encoding='latin-1')
sms_df.columns = ['class', 'sms', 'a', 'b', 'c']
sms_df = sms_df.drop(['a', 'b', 'c'], axis=1)
sms_df = sms_df.apply(lambda x: x.astype(str).str.lower())


In [181]:
from sklearn.feature_extraction.text import CountVectorizer
my_vec = CountVectorizer(decode_error = 'ignore', stop_words = 'english')
X = my_vec.fit_transform(sms_df['sms'].values)


In [182]:
top_spam = top_k(sms_df[sms_df['class'] == 'spam'], my_vec, 30)
top_ham = top_k(sms_df[sms_df['class'] == 'ham'], my_vec, 30)
from regularize import regularize_urls
from regularize import regularize_numbers
sms_df['sms'] = regularize_numbers(sms_df['sms'])
sms_df['sms'] = regularize_urls(sms_df['sms'])

In [183]:
# Top-30 spam terms
from sklearn.feature_extraction.text import CountVectorizer
my_vec = CountVectorizer(decode_error = 'ignore', stop_words = 'english')
X = my_vec.fit_transform(sms_df['sms'].values)

top_ham = top_k(sms_df[sms_df['class'] == 'ham'], my_vec, 30)
top_spam = top_k(sms_df[sms_df['class'] == 'spam'], my_vec, 30)

top_ham_vocab = list(top_ham.keys())
top_spam_vocab = list(top_spam.keys())
total_top_vocab = list(set(top_ham_vocab + top_spam_vocab))
relevant_vec = CountVectorizer(vocabulary = total_top_vocab,decode_error = 'ignore', stop_words = 'english')
X = relevant_vec.fit_transform(sms_df['sms'].values).toarray()



In [184]:
import sklearn.model_selection as ms
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
import numpy as np

class_le = LabelEncoder() 

# X is the feature array, based off relevant words
X = relevant_vec.fit_transform(sms_df['sms'].values).toarray()

# Compute the length of each sms message, normalized
# by max length
Xlen = np.zeros((X.shape[0],1))
inx = 0
for v in sms_df['sms'].values:
        Xlen[inx,0] = len(v)
        inx += 1
Xlen = Xlen / max(Xlen)
# Add the length as another feature
X = np.hstack((X, Xlen))

y = sms_df['class'].values
#y = class_le.fit_transform(sms_df['class'].values)

# Now we split...
X_train, X_test, y_train, y_test = ms.train_test_split(X, 
                                                    y, test_size=0.2)

X_train

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.01536773],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.11964874],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.31174533],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.32821076],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.05049396],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.11745335]])

In [176]:
y_train = class_le.fit_transform(y_train)

In [32]:
len(X)

5572

In [185]:
#feature_columns = [tf.contrib.layers.real_valued_column("", dimension=57)]
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,hidden_units=[5,5])

In [186]:
def input_fn(x,y):
    tx = tf.constant(x)
    ty = tf.constant(y)
    return tx,ty

def test_input_fn():
    x,y = input_fn(X_test,y_test)
    return x,y
    
def train_input_fn():
    x,y = input_fn(X_train,y_train)
    return x,y

In [187]:
classifier.fit(input_fn=train_input_fn, steps=1000)



TypeError: only integer scalar arrays can be converted to a scalar index

In [147]:
#x,y = train_input_fn()
#x
#y
y_train = np.ones((4457), dtype=np.float64)
x_train = np.ones((4457,57),dtype=np.float64)
#y_train = y_train.astype(int)

In [154]:
#y_train = np.array(y_train, dtype=np.int) 
classifier.fit(x=X_train, y=y_train, steps=1000)
#classifier.fit(x=training_set.data, y=training_set.target, steps=1000)


TypeError: only integer scalar arrays can be converted to a scalar index

In [133]:
y_train.shape

AttributeError: 'Tensor' object has no attribute 'shape'

In [80]:
np.isscalar(y_train)

False

In [85]:

y_train = np.array(y_train, dtype=np.int) 

In [122]:
import tensorflow.contrib.learn.python.learn.datasets

In [125]:
sess = tf.InteractiveSession()

In [126]:
x = tf.placeholder(tf.float32,[None,784])

In [188]:
y = tf.placeholder(tf.float32,[None,10])

In [189]:
np.savetxt('X_training.txt',X_train, delimiter=',')

In [191]:
np.savetxt('X_training.txt',X_train, delimiter=',',fmt='%f')

In [204]:
y_train = class_le.fit_transform(y_train)

In [195]:
np.savetxt('y_training.txt',y_train, delimiter=',',fmt='%d')

In [198]:
train = np.column_stack((X_train,y_train))

In [201]:
np.savetxt('train.txt',train, delimiter=',',fmt='%f')

In [202]:
y_test

array(['spam', 'ham', 'ham', ..., 'ham', 'ham', 'spam'], dtype=object)

In [205]:
y_test = class_le.fit_transform(y_test)

In [206]:
test = np.column_stack((X_test,y_test))

In [216]:
train.shape

(4457, 58)

In [207]:
np.savetxt('test.txt',test, delimiter=',',fmt='%f')

In [232]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib

import numpy as np
import tensorflow as tf


def main():
    TEST = "test6.txt"
    test_set = tf.contrib.learn.datasets.base.load_csv(
            filename=TEST,
            target_dtype=np.int
            )
    TRAIN = "train6.txt"
    training_set = tf.contrib.learn.datasets.base.load_csv(
            filename=TRAIN,
            target_dtype=np.int
            )
    # Specify that all features have real-value data
    feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

    # Build 3 layer DNN with 5,5 units respectively.
    classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                              hidden_units=[10, 20,20],
                                              n_classes=3,
                                              model_dir="/tmp/iris_model")
    # Define the training inputs
    def get_train_inputs():
        x = tf.constant(training_set.data)
        y = tf.constant(training_set.target)

        return x, y

    # Fit model.
    classifier.fit(input_fn=get_train_inputs, steps=2000)

    # Define the test inputs
    def get_test_inputs():
        x = tf.constant(test_set.data)
        y = tf.constant(test_set.target)

        return x, y

    # Evaluate accuracy.
    accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                       steps=1)["accuracy"]
    
    print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

if __name__ == "__main__":
    main()  

Instructions for updating:
Please use load_csv_{with|without}_header instead.
Instructions for updating:
Please use load_csv_{with|without}_header instead.


InvalidArgumentError: Shape in shape_and_slice spec does not match the shape in the save file: [20,3], save file shape: [10,3]
	 [[Node: save/restore_slice_17 = RestoreSlice[dt=DT_FLOAT, preferred_shard=0, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/restore_slice_17/tensor_name, save/restore_slice_17/shape_and_slice)]]
Caused by op 'save/restore_slice_17', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/nintendo/cis700/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/nintendo/cis700/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/nintendo/cis700/lib/python3.5/site-pac